In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/kobo2020'
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/kobo2020
AttentionSeq2seq.pkl                neko.txt
attention_seq2seq.py                nlp100_chap9
BetterRnnlm.pkl                     nlp100_chap9.pkl
better_rnnlm.py                     nlp100_chap9.txt
cbow_params.pkl                     __pycache__/
cbow.py                             PyTorchTutorial1.ipynb
ch04/                               repo10.ipynb
ch06/                               repo10test.ipynb
ch0

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pack_padded_sequence, pad_sequence
import re
from collections import Counter
import pickle

ja_word2id_path = "jaword2id.pickle"
en_word2id_path = "enword2id.pickle"
ja_word2id = pickle.load(open(ja_word2id_path, "rb"))
en_word2id = pickle.load(open(en_word2id_path, "rb"))

#言語に応じたデータを取り出してidのリストを返す
def get_ids(word_list, lang):
    if lang == "ja":
        word2id = ja_word2id
    else:
        word2id = en_word2id

    ids = []
    for word in word_list:
        if word in word2id:
            ids.append(word2id[word])
        else:
            ids.append(word2id["[UNK]"])
    return ids


# 使用するデータの英語と日本語の文に応じたid列のテンソル が入ったリストを返す
def get_data(ja_fname, en_fname):
    ja_ids_list = []
    en_ids_list = []

    with open(ja_fname) as f:
        for line in f:
            if not line:
                continue
            word_list = line.split()
            ids = get_ids(word_list, "ja")
            ja_ids_list.append(torch.tensor(ids))

    with open(en_fname) as f:
        for line in f:
            if not line:
                continue
            word_list = line.split()
            #print(word_list)
            ids = get_ids(word_list, "en")
            en_ids_list.append(torch.tensor(ids))

    return ja_ids_list, en_ids_list


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pack_padded_sequence, pad_sequence

ja_data, en_data = get_data("small_parallel_enja/train.ja", "small_parallel_enja/train.en")

ja_one_text = ja_data[0]
en_one_text = ja_data[0]
print(ja_one_text)
emb = nn.Embedding(10000,5) #辞書の単語数 ,隠れ層のサイズ 
#print(len(ja_one_text))#16
out = emb(ja_one_text)#例) 
#print(out.shape)#16x5
print(out)#単語ベクトルを隠れ層のサイズに合わせて生成？　テンソルの中の一行が一単語に対応？


tensor([ 91,  13, 229,   6, 154,  28,  21,  17,   6,   4, 276,  37,  20,  40,
         29,   3])
tensor([[ 1.3531,  1.6825,  0.3928,  1.0511,  1.6332],
        [-0.1801,  1.9428, -1.1983, -0.1217, -0.6933],
        [-0.0738,  0.2610, -1.4792, -1.3616,  0.4303],
        [-1.3940,  0.6831,  0.1174, -1.0609, -0.8274],
        [ 0.1608,  0.4608, -1.5504, -0.1069,  1.7522],
        [-0.1669, -0.3400,  0.8054, -0.3359, -0.8326],
        [ 0.3873,  0.2671,  0.7411,  0.8028,  0.8499],
        [ 0.3686,  0.1633, -0.1061, -0.8438, -2.0437],
        [-1.3940,  0.6831,  0.1174, -1.0609, -0.8274],
        [ 0.6816,  1.3218,  0.1829, -0.4052, -1.3517],
        [-1.7152,  2.2548, -1.7128, -1.2368, -1.3583],
        [ 0.6295, -1.7423, -1.3730,  1.4075, -0.9732],
        [ 0.3860, -0.8883, -0.9921,  0.5602, -1.5130],
        [-0.9949, -0.9064, -1.7262, -0.2596,  1.1090],
        [ 1.1538,  2.4834,  1.7380,  0.7371, -0.0558],
        [ 1.0355,  0.3189,  0.5838,  0.3522,  0.3494]],
       grad_fn=<Embedd

In [32]:
a = ja_data[0]
b = ja_data[1]
ja_two_text = torch.cat([a,b],dim = 0)

print(ja_two_text,ja_two_text.shape)#文くっつけちゃったから30単語のランダムな単語ベクトルが出力されている？？
out2 = emb(ja_two_text)
print(out2,out2.shape)

tensor([ 91,  13, 229,   6, 154,  28,  21,  17,   6,   4, 276,  37,  20,  40,
         29,   3, 287,   9, 700,  13, 701,   6,  34,  19,  10,   2,  51,  53,
          7,   3]) torch.Size([30])
tensor([[ 1.3531,  1.6825,  0.3928,  1.0511,  1.6332],
        [-0.1801,  1.9428, -1.1983, -0.1217, -0.6933],
        [-0.0738,  0.2610, -1.4792, -1.3616,  0.4303],
        [-1.3940,  0.6831,  0.1174, -1.0609, -0.8274],
        [ 0.1608,  0.4608, -1.5504, -0.1069,  1.7522],
        [-0.1669, -0.3400,  0.8054, -0.3359, -0.8326],
        [ 0.3873,  0.2671,  0.7411,  0.8028,  0.8499],
        [ 0.3686,  0.1633, -0.1061, -0.8438, -2.0437],
        [-1.3940,  0.6831,  0.1174, -1.0609, -0.8274],
        [ 0.6816,  1.3218,  0.1829, -0.4052, -1.3517],
        [-1.7152,  2.2548, -1.7128, -1.2368, -1.3583],
        [ 0.6295, -1.7423, -1.3730,  1.4075, -0.9732],
        [ 0.3860, -0.8883, -0.9921,  0.5602, -1.5130],
        [-0.9949, -0.9064, -1.7262, -0.2596,  1.1090],
        [ 1.1538,  2.4834,  1.7380,  0

In [40]:
ja_two_tensor = torch.tensor([[91,13,229,6,154,28,21,17,6,4,3],[1,2,3,4,5,6,7,8,9,10,11]])#二文をまとめたバッチ？ 　　2x11 文の数x単語数
print(ja_two_tensor,ja_two_tensor.shape)
out2 = emb(ja_two_tensor)
print(out2,out2.shape)#2x11x5　で、
"""
[単語, 単語,単語, ... ,単語]←文 [[単語, 単語,単語, ... ,単語],[単語, 単語,単語, ... ,単語]] ←文x2を
[[[1xHの形状のランダムな単語ベクトル], [1xHの形状のランダムな単語ベクトル], ... ,[1xHの形状のランダムな単語ベクトル]],[[1xHの形状のランダムな単語ベクトル], [1xHの形状のランダムな単語ベクトル], ... ,[1xHの形状のランダムな単語ベクトル]]]
に変換してる？
"""


tensor([[ 91,  13, 229,   6, 154,  28,  21,  17,   6,   4,   3],
        [  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11]]) torch.Size([2, 11])
tensor([[[ 1.3531,  1.6825,  0.3928,  1.0511,  1.6332],
         [-0.1801,  1.9428, -1.1983, -0.1217, -0.6933],
         [-0.0738,  0.2610, -1.4792, -1.3616,  0.4303],
         [-1.3940,  0.6831,  0.1174, -1.0609, -0.8274],
         [ 0.1608,  0.4608, -1.5504, -0.1069,  1.7522],
         [-0.1669, -0.3400,  0.8054, -0.3359, -0.8326],
         [ 0.3873,  0.2671,  0.7411,  0.8028,  0.8499],
         [ 0.3686,  0.1633, -0.1061, -0.8438, -2.0437],
         [-1.3940,  0.6831,  0.1174, -1.0609, -0.8274],
         [ 0.6816,  1.3218,  0.1829, -0.4052, -1.3517],
         [ 1.0355,  0.3189,  0.5838,  0.3522,  0.3494]],

        [[-0.9075, -0.6647, -0.6396, -0.2718,  0.7547],
         [-1.4652, -0.0392, -0.4217, -1.5935, -0.0847],
         [ 1.0355,  0.3189,  0.5838,  0.3522,  0.3494],
         [ 0.6816,  1.3218,  0.1829, -0.4052, -1.3517],
       

In [30]:
x = torch.tensor([[1, 2], [3, 4.]])
print(x)

tensor([[1., 2.],
        [3., 4.]])
